In [35]:
'''
Deep MNIST
'''

'\nDeep MNIST\n'

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".",one_hot = True,reshape = False)
import tensorflow as tf

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [3]:
#parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

test_valid_size = 256

n_classes = 10
dropout = 0.75

In [16]:
# Weights and Bias

weights = {
    'wc1':tf.Variable(tf.random_normal([5,5,1,32])),
    'wc2':tf.Variable(tf.random_normal([5,5,32,64])),
    'wd1':tf.Variable(tf.random_normal([7*7*64,1024])),
    'out':tf.Variable(tf.random_normal([1024,n_classes])),
}

biases = {
    'bc1':tf.Variable(tf.random_normal([32])),
    'bc2':tf.Variable(tf.random_normal([64])),
    'bd1':tf.Variable(tf.random_normal([1024])),
    'out':tf.Variable(tf.random_normal([n_classes])),
}


In [23]:
#Layers
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)
def maxpool2d(x,k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

In [32]:
#Model
def conv_net(x, weights, biases, dropout):

    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)


    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [33]:
#Session


x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

logits = conv_net(x, weights, biases, keep_prob)

cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


init = tf. global_variables_initializer()


with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

  
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))
  

Epoch  1, Batch   1 -Loss: 48277.3047 Validation Accuracy: 0.152344
Epoch  1, Batch   2 -Loss: 37880.5469 Validation Accuracy: 0.144531
Epoch  1, Batch   3 -Loss: 34515.8672 Validation Accuracy: 0.156250
Epoch  1, Batch   4 -Loss: 25352.6426 Validation Accuracy: 0.160156
Epoch  1, Batch   5 -Loss: 27838.9941 Validation Accuracy: 0.183594
Epoch  1, Batch   6 -Loss: 25105.3438 Validation Accuracy: 0.179688
Epoch  1, Batch   7 -Loss: 23509.3672 Validation Accuracy: 0.187500
Epoch  1, Batch   8 -Loss: 25991.5000 Validation Accuracy: 0.183594
Epoch  1, Batch   9 -Loss: 18188.0469 Validation Accuracy: 0.175781
Epoch  1, Batch  10 -Loss: 18868.7676 Validation Accuracy: 0.203125
Epoch  1, Batch  11 -Loss: 16550.7070 Validation Accuracy: 0.214844
Epoch  1, Batch  12 -Loss: 16441.7031 Validation Accuracy: 0.218750
Epoch  1, Batch  13 -Loss: 14448.8535 Validation Accuracy: 0.226562
Epoch  1, Batch  14 -Loss: 14704.6641 Validation Accuracy: 0.226562
Epoch  1, Batch  15 -Loss: 15877.4160 Validation

Epoch  1, Batch 122 -Loss:  3833.4062 Validation Accuracy: 0.589844
Epoch  1, Batch 123 -Loss:  4590.0815 Validation Accuracy: 0.593750
Epoch  1, Batch 124 -Loss:  3982.7900 Validation Accuracy: 0.589844
Epoch  1, Batch 125 -Loss:  3281.1289 Validation Accuracy: 0.585938
Epoch  1, Batch 126 -Loss:  4514.1919 Validation Accuracy: 0.585938
Epoch  1, Batch 127 -Loss:  3591.3738 Validation Accuracy: 0.593750
Epoch  1, Batch 128 -Loss:  3764.1201 Validation Accuracy: 0.601562
Epoch  1, Batch 129 -Loss:  4053.5920 Validation Accuracy: 0.605469
Epoch  1, Batch 130 -Loss:  3394.7388 Validation Accuracy: 0.601562
Epoch  1, Batch 131 -Loss:  3450.9805 Validation Accuracy: 0.613281
Epoch  1, Batch 132 -Loss:  3290.9727 Validation Accuracy: 0.609375
Epoch  1, Batch 133 -Loss:  3601.3113 Validation Accuracy: 0.609375
Epoch  1, Batch 134 -Loss:  4097.8760 Validation Accuracy: 0.617188
Epoch  1, Batch 135 -Loss:  3032.7700 Validation Accuracy: 0.609375
Epoch  1, Batch 136 -Loss:  2989.5164 Validation

Epoch  1, Batch 243 -Loss:  1799.5906 Validation Accuracy: 0.699219
Epoch  1, Batch 244 -Loss:  1875.1307 Validation Accuracy: 0.707031
Epoch  1, Batch 245 -Loss:  2128.0374 Validation Accuracy: 0.703125
Epoch  1, Batch 246 -Loss:  3389.4937 Validation Accuracy: 0.703125
Epoch  1, Batch 247 -Loss:  2833.2385 Validation Accuracy: 0.707031
Epoch  1, Batch 248 -Loss:  2947.3225 Validation Accuracy: 0.695312
Epoch  1, Batch 249 -Loss:  1779.5195 Validation Accuracy: 0.714844
Epoch  1, Batch 250 -Loss:  2567.0229 Validation Accuracy: 0.714844
Epoch  1, Batch 251 -Loss:  2982.9551 Validation Accuracy: 0.710938
Epoch  1, Batch 252 -Loss:  2227.0137 Validation Accuracy: 0.699219
Epoch  1, Batch 253 -Loss:  2527.4194 Validation Accuracy: 0.703125
Epoch  1, Batch 254 -Loss:  2185.1738 Validation Accuracy: 0.695312
Epoch  1, Batch 255 -Loss:  1834.4031 Validation Accuracy: 0.695312
Epoch  1, Batch 256 -Loss:  2921.4524 Validation Accuracy: 0.703125
Epoch  1, Batch 257 -Loss:  2810.4272 Validation

Epoch  1, Batch 364 -Loss:  1811.8176 Validation Accuracy: 0.707031
Epoch  1, Batch 365 -Loss:  1706.4620 Validation Accuracy: 0.707031
Epoch  1, Batch 366 -Loss:  1745.0842 Validation Accuracy: 0.710938
Epoch  1, Batch 367 -Loss:  1813.8807 Validation Accuracy: 0.714844
Epoch  1, Batch 368 -Loss:  1845.8223 Validation Accuracy: 0.707031
Epoch  1, Batch 369 -Loss:  1267.8911 Validation Accuracy: 0.718750
Epoch  1, Batch 370 -Loss:  1535.2026 Validation Accuracy: 0.722656
Epoch  1, Batch 371 -Loss:  1852.7014 Validation Accuracy: 0.726562
Epoch  1, Batch 372 -Loss:  2021.7926 Validation Accuracy: 0.722656
Epoch  1, Batch 373 -Loss:  1601.9022 Validation Accuracy: 0.722656
Epoch  1, Batch 374 -Loss:  2295.0742 Validation Accuracy: 0.722656
Epoch  1, Batch 375 -Loss:  1952.3071 Validation Accuracy: 0.722656
Epoch  1, Batch 376 -Loss:  1861.1899 Validation Accuracy: 0.718750
Epoch  1, Batch 377 -Loss:  1834.2516 Validation Accuracy: 0.718750
Epoch  1, Batch 378 -Loss:  1704.0039 Validation

Epoch  2, Batch  56 -Loss:  1251.8290 Validation Accuracy: 0.726562
Epoch  2, Batch  57 -Loss:  1457.0625 Validation Accuracy: 0.730469
Epoch  2, Batch  58 -Loss:  1711.8258 Validation Accuracy: 0.722656
Epoch  2, Batch  59 -Loss:  1120.7729 Validation Accuracy: 0.722656
Epoch  2, Batch  60 -Loss:  1213.1609 Validation Accuracy: 0.730469
Epoch  2, Batch  61 -Loss:  1295.6598 Validation Accuracy: 0.734375
Epoch  2, Batch  62 -Loss:   864.6252 Validation Accuracy: 0.738281
Epoch  2, Batch  63 -Loss:  1149.8784 Validation Accuracy: 0.726562
Epoch  2, Batch  64 -Loss:  1329.9561 Validation Accuracy: 0.722656
Epoch  2, Batch  65 -Loss:  1144.5991 Validation Accuracy: 0.722656
Epoch  2, Batch  66 -Loss:  1090.2681 Validation Accuracy: 0.718750
Epoch  2, Batch  67 -Loss:  1235.4985 Validation Accuracy: 0.714844
Epoch  2, Batch  68 -Loss:  1280.9575 Validation Accuracy: 0.714844
Epoch  2, Batch  69 -Loss:  2041.5364 Validation Accuracy: 0.726562
Epoch  2, Batch  70 -Loss:  1192.6156 Validation

...

Epoch  9, Batch 320 -Loss:   183.1537 Validation Accuracy: 0.816406
Epoch  9, Batch 321 -Loss:   280.3081 Validation Accuracy: 0.820312
Epoch  9, Batch 322 -Loss:   322.2628 Validation Accuracy: 0.820312
Epoch  9, Batch 323 -Loss:   248.5318 Validation Accuracy: 0.828125
Epoch  9, Batch 324 -Loss:   272.1298 Validation Accuracy: 0.828125
Epoch  9, Batch 325 -Loss:   224.9351 Validation Accuracy: 0.828125
Epoch  9, Batch 326 -Loss:   219.9164 Validation Accuracy: 0.820312
Epoch  9, Batch 327 -Loss:   174.9606 Validation Accuracy: 0.820312
Epoch  9, Batch 328 -Loss:   245.4153 Validation Accuracy: 0.824219
Epoch  9, Batch 329 -Loss:   226.0587 Validation Accuracy: 0.824219
Epoch  9, Batch 330 -Loss:   185.1626 Validation Accuracy: 0.824219
Epoch  9, Batch 331 -Loss:   296.2958 Validation Accuracy: 0.820312
Epoch  9, Batch 332 -Loss:   123.4411 Validation Accuracy: 0.816406
Epoch  9, Batch 333 -Loss:   197.3092 Validation Accuracy: 0.816406
Epoch  9, Batch 334 -Loss:   282.1439 Validation

Epoch 10, Batch  12 -Loss:   230.8634 Validation Accuracy: 0.824219
Epoch 10, Batch  13 -Loss:   193.2487 Validation Accuracy: 0.828125
Epoch 10, Batch  14 -Loss:   265.9307 Validation Accuracy: 0.828125
Epoch 10, Batch  15 -Loss:   212.7885 Validation Accuracy: 0.832031
Epoch 10, Batch  16 -Loss:   166.6488 Validation Accuracy: 0.828125
Epoch 10, Batch  17 -Loss:   350.4139 Validation Accuracy: 0.832031
Epoch 10, Batch  18 -Loss:   217.7729 Validation Accuracy: 0.828125
Epoch 10, Batch  19 -Loss:   197.0757 Validation Accuracy: 0.820312
Epoch 10, Batch  20 -Loss:   256.5115 Validation Accuracy: 0.816406
Epoch 10, Batch  21 -Loss:   230.6174 Validation Accuracy: 0.816406
Epoch 10, Batch  22 -Loss:   335.7977 Validation Accuracy: 0.816406
Epoch 10, Batch  23 -Loss:   177.7925 Validation Accuracy: 0.820312
Epoch 10, Batch  24 -Loss:   268.2518 Validation Accuracy: 0.816406
Epoch 10, Batch  25 -Loss:   299.7511 Validation Accuracy: 0.824219
Epoch 10, Batch  26 -Loss:   209.8697 Validation

Epoch 10, Batch 133 -Loss:   326.0005 Validation Accuracy: 0.816406
Epoch 10, Batch 134 -Loss:   250.3477 Validation Accuracy: 0.816406
Epoch 10, Batch 135 -Loss:   203.0744 Validation Accuracy: 0.824219
Epoch 10, Batch 136 -Loss:   381.6980 Validation Accuracy: 0.816406
Epoch 10, Batch 137 -Loss:   249.7668 Validation Accuracy: 0.816406
Epoch 10, Batch 138 -Loss:   350.9388 Validation Accuracy: 0.820312
Epoch 10, Batch 139 -Loss:   175.0396 Validation Accuracy: 0.820312
Epoch 10, Batch 140 -Loss:   166.5083 Validation Accuracy: 0.820312
Epoch 10, Batch 141 -Loss:   217.3579 Validation Accuracy: 0.816406
Epoch 10, Batch 142 -Loss:   246.1240 Validation Accuracy: 0.816406
Epoch 10, Batch 143 -Loss:   217.8396 Validation Accuracy: 0.816406
Epoch 10, Batch 144 -Loss:   362.5135 Validation Accuracy: 0.820312
Epoch 10, Batch 145 -Loss:   229.1197 Validation Accuracy: 0.816406
Epoch 10, Batch 146 -Loss:   322.3518 Validation Accuracy: 0.804688
Epoch 10, Batch 147 -Loss:   122.2236 Validation

Epoch 10, Batch 254 -Loss:   218.4073 Validation Accuracy: 0.820312
Epoch 10, Batch 255 -Loss:   194.9857 Validation Accuracy: 0.820312
Epoch 10, Batch 256 -Loss:   220.8866 Validation Accuracy: 0.820312
Epoch 10, Batch 257 -Loss:   257.8473 Validation Accuracy: 0.812500
Epoch 10, Batch 258 -Loss:   236.4477 Validation Accuracy: 0.824219
Epoch 10, Batch 259 -Loss:   211.1346 Validation Accuracy: 0.824219
Epoch 10, Batch 260 -Loss:   190.7436 Validation Accuracy: 0.820312
Epoch 10, Batch 261 -Loss:   205.3265 Validation Accuracy: 0.820312
Epoch 10, Batch 262 -Loss:   364.6397 Validation Accuracy: 0.820312
Epoch 10, Batch 263 -Loss:   180.9342 Validation Accuracy: 0.812500
Epoch 10, Batch 264 -Loss:   152.8575 Validation Accuracy: 0.816406
Epoch 10, Batch 265 -Loss:   228.1093 Validation Accuracy: 0.816406
Epoch 10, Batch 266 -Loss:   358.0479 Validation Accuracy: 0.808594
Epoch 10, Batch 267 -Loss:   270.6005 Validation Accuracy: 0.808594
Epoch 10, Batch 268 -Loss:   148.6741 Validation

Epoch 10, Batch 375 -Loss:   200.2264 Validation Accuracy: 0.824219
Epoch 10, Batch 376 -Loss:   264.8220 Validation Accuracy: 0.820312
Epoch 10, Batch 377 -Loss:   187.5351 Validation Accuracy: 0.820312
Epoch 10, Batch 378 -Loss:   153.4865 Validation Accuracy: 0.820312
Epoch 10, Batch 379 -Loss:   217.8749 Validation Accuracy: 0.824219
Epoch 10, Batch 380 -Loss:   134.5247 Validation Accuracy: 0.820312
Epoch 10, Batch 381 -Loss:   247.0560 Validation Accuracy: 0.820312
Epoch 10, Batch 382 -Loss:   257.7809 Validation Accuracy: 0.820312
Epoch 10, Batch 383 -Loss:   169.0218 Validation Accuracy: 0.820312
Epoch 10, Batch 384 -Loss:   190.9815 Validation Accuracy: 0.820312
Epoch 10, Batch 385 -Loss:   226.4414 Validation Accuracy: 0.824219
Epoch 10, Batch 386 -Loss:   199.6525 Validation Accuracy: 0.820312
Epoch 10, Batch 387 -Loss:   247.5534 Validation Accuracy: 0.820312
Epoch 10, Batch 388 -Loss:   239.2239 Validation Accuracy: 0.816406
Epoch 10, Batch 389 -Loss:   234.9576 Validation

In [34]:
print('Testing Accuracy: {}'.format(test_acc))

Testing Accuracy: 0.77734375
